In [2]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import re
import tensorflow as tf
import numpy as np

In [3]:
def display_images(images, n_line, l_col):
    plt.figure(figsize=(10, 10))
    for i in range(n_line*l_col):
        ax = plt.subplot(n_line, l_col, i + 1)
        plt.imshow(images[i])
        plt.axis("off")

In [4]:
def build_boxes(x, y, w, h):
    N = 5
    shift = list(range(-N, N+1, 2))
    
    pos = []
    for i in shift:
        new_x = x + i
        
        for j in shift:
            new_y = y + j
            
            new_pos = (new_x, new_y, w, h)
            pos.append(new_pos)
            
            # Zoom modification
            zoom = [1.2, 0.8]
            for z in zoom:
                new_w, new_h = int(w*z), int(h*z)
                new_x = new_x - (abs(new_w - w) // 2)
                new_y = new_y - (abs(new_h - h) // 2)
                
                pos.append((new_x, new_y, new_w, new_h))
            
    return pos

In [5]:
def read_bounding_box(path):
    boxes = {}
    m = re.search('VIDEOS/(.+)\.mp4', path)
    filename = m.group(1)
    
    f = open(f'../GT/{filename}_2_bboxes.txt', 'r')
    lines = f.readlines()
    
    for l in lines:
        elmts = l.split(' ')
        frame = int(elmts[0])
        
        if elmts[1] == '1':
            boxes[frame] = (int(elmts[2]), int(elmts[3]), int(elmts[4]), int(elmts[5]))
            
    return boxes

In [6]:
model = tf.keras.models.load_model('../models/custom_13_12_2020', custom_objects=None, compile=True, options=None)

In [46]:
labels = { 0: 'bol', 1: 'coca', 2: 'lait', 3: 'riz', 4: 'sucre' }
def get_images(filename):
    predict_boxes = {}
    
    # on récupère les boundings box du fichier
    boxes = read_bounding_box(filename)
    
    # on ouvre le fichier
    cap = cv2.VideoCapture(filename)
    nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if(not cap.isOpened()):
        print("ERROR: unable to read video:", video_filename)
        sys.exit()
    
    print(f'Loading {nframes} frames')
    print(len(boxes))
    
    if 'CanOfCocaCola' in filename:
        label = 1
    elif 'MilkBottle' in filename:
        label = 2
    elif 'Bowl' in filename:
        label = 0
    elif 'Rice' in filename:
        label = 3
    elif 'Sugar' in filename:
        label = 4
    
    foundObj = False
    for f in range(nframes):
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # conversion BRG to RGB
        
        if f in boxes: # on ne regarde que les frames qui ont une bounding box
            if not foundObj:
                box = boxes[f] # première bounding box (= box de référence)
            else:
                x, y, w, h = box
                
                # on créé de nouvelles bouding box autour de celle de la frame précédente
                new_boxes = build_boxes(x, y, w, h)
                #patch = frame[y:y+h, x:x+w]
                
                # pour chaque nouvel box, on extrait le patch associé dans l'image
                patches = np.zeros((0, 227, 227, 3))
                for bb in new_boxes:
                    x1, y1, w1, h1 = bb
                    #new_patch = tf.expand_dims(frame[y1:y1+h1, x1:x1+w1, :], 0) 
                    new_patch = frame[y1:y1+h1, x1:x1+w1, :]
                    new_patch = tf.image.resize(new_patch, (227, 227), method='nearest')
                    patches = tf.concat((patches, tf.expand_dims(new_patch, 0)), axis=0)
                
                # on passe les patchs dans le modèle
                pred = model.predict(patches)
                
                # on récupère la meilleure prédiction
                best_pred = np.argmax(pred[:, label])
                
                # on récupère le patch associé à la meilleure prédiction
                best_patch = patches[best_pred]
                best_box = new_boxes[best_pred]
                
                # on modifie la current box pour la frame suivante
                box = best_box
                predict_boxes[f] = best_box
                
                """if f == 26:
                    a, b, c, d = best_box
                    p = frame[b:b+d, a:a+c, :]
                    plt.imshow(p)
                    break"""
                    
            foundObj = True
    
    return predict_boxes

In [47]:
tracking = get_images('../VIDEOS/CanOfCocaColaPlace6Subject4.mp4')

Loading 186 frames
163


InvalidArgumentError: input image must be of non-zero size [Op:ResizeNearestNeighbor]